<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter04/Time_comparison_of_augmentation_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets
import torch
data_folder = '../Chapter03/' # This can be any directory you want to download FMNIST to
fmnist = datasets.FashionMNIST(data_folder, download=False, train=True)

In [2]:
tr_images = fmnist.data.numpy()
tr_targets = fmnist.targets

In [3]:
val_fmnist = datasets.FashionMNIST(data_folder, download=False, train=False)
val_images = val_fmnist.data.numpy()
val_targets = val_fmnist.targets

In [4]:
from imgaug import augmenters as iaa

aug = iaa.Sequential([
              iaa.Affine(translate_px={'x':(-10,10)}, mode='constant'),
            ])

In [6]:
%%time
for i in range(32):
  aug.augment_image(tr_images[i])

CPU times: user 21.3 ms, sys: 81 μs, total: 21.3 ms
Wall time: 20.4 ms


In [9]:
%%time
x = aug.augment_images(tr_images[:32])
x.shape

CPU times: user 163 ms, sys: 7.96 ms, total: 171 ms
Wall time: 168 ms


(32, 28, 28)

In [10]:
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class FMNISTDataset(Dataset):
    def __init__(self, images, targets, aug=None):
        self.images = images
        self.targets = targets
        self.aug = aug

    def __getitem__(self, index):
        x, y = self.images[index], self.targets[index]
        return x, y

    def __len__(self):
        return len(self.images)

    def collate_fn(self, batch):
        img, classes = list(zip(*batch))
        if self.aug:
            img = self.aug.augment_images(img)
        img = torch.tensor(img)[:, None, :, :].to(device) / 255.
        classes = torch.tensor(classes).to(device)
        return img, classes

In [ ]:
train = FMNISTDataset(tr_images, tr_targets, aug=aug)
trn_dl = DataLoader(train, batch_size=64, collate_fn=train.collate_fn, shuffle=True)
